In [ ]:
!apt-get update
!apt-get install -y \
    libatk1.0-0 \
    libatk-bridge2.0-0 \
    libcups2 \
    libxcomposite1 \
    libxrandr2 \
    libxdamage1 \
    libxkbcommon0 \
    libgbm1 \
    libpango-1.0-0 \
    libpangocairo-1.0-0 \
    libgtk-3-0 \
    libnss3 \
    libasound2

In [9]:
!pip install playwright pandas nest_asyncio
!playwright install chromium

In [10]:
import asyncio
import pandas as pd
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

BASE_URL = "https://quotes.toscrape.com/js"

async def scrape_quotes():
    quotes_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(BASE_URL)

        while True:
            await page.wait_for_selector("div.quote")

            quotes = await page.query_selector_all("div.quote")

            for quote in quotes:
                text = await quote.query_selector("span.text")
                author = await quote.query_selector("small.author")
                tags = await quote.query_selector_all("a.tag")

                quote_text = await text.inner_text()
                author_name = await author.inner_text()
                tag_list = [await tag.inner_text() for tag in tags]

                quotes_data.append({
                    "Quote": quote_text,
                    "Author": author_name,
                    "Tags": ", ".join(tag_list)
                })

            # Handle pagination
            next_button = await page.query_selector("li.next > a")
            if next_button:
                await next_button.click()
                await page.wait_for_load_state("networkidle")
            else:
                break

        await browser.close()

    return quotes_data


# Run async scraper
data = asyncio.run(scrape_quotes())

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("quotes.csv", index=False)

print("Scraping complete! CSV saved as quotes.csv")


Scraping complete! CSV saved as quotes.csv
